In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from pathlib import Path
from keras.layers import Input,Conv2D,Dense,Flatten,Dropout
from keras.layers import GlobalMaxPooling2D,MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
import pandas_datareader as pdr
import pandas as pd
import pandas as pd 
import numpy as np 
import os
from pathlib import Path



In [ ]:
with_mask_dir = Path('../input/face-mask-dataset/FMD_DATASET/with_mask/simple')
mc_dir= Path('../input/face-mask-dataset/FMD_DATASET/incorrect_mask/mc')
mmc_dir= Path('../input/face-mask-dataset/FMD_DATASET/incorrect_mask/mmc')
without_mask_dir= Path('../input/face-mask-dataset/FMD_DATASET/without_mask/simple')

In [ ]:
with_mask_dir.glob(r'*jpg')
without_mask_dir.glob(r'*jpg')
mc_dir.glob(r'*jpg')
mmc_dir.glob(r'*jpg')


<generator object Path.glob at 0x7f4872d39a50>

In [ ]:
def generate_df(image_dir, label):
    filepath = pd.Series(list(image_dir.glob(r'*jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepath.index).astype(str)
    df = pd.concat([filepath, labels], axis=1)
    return df

In [ ]:
with_mask_df = generate_df(with_mask_dir, label=0)
without_mask_df = generate_df(without_mask_dir, label=1)
mc_df = generate_df(mc_dir, label=2)
mmc_df = generate_df(mmc_dir, label=3)
total_df = pd.concat([with_mask_df, without_mask_df, mc_df, mmc_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
total_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,Filepath,Label


In [ ]:
total_df.Label.value_counts(normalize=True)




Series([], Name: Label, dtype: float64)

In [ ]:
from sklearn.model_selection import train_test_split
#The dataset  is now imported from keras datasets.
#Now we have to load and distribute the data to train and test set 
train_df, test_df = train_test_split(total_df,train_size=0.7,shuffle=True, random_state=1)


ValueError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
      rescale=1./255,
      validation_split=.2,
      rotation_range=40,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest'
)

test_gen = ImageDataGenerator(
      rescale=1./255,
)

train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_data = test_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=42
)


In [ ]:
import matplotlib.pyplot as plt 
from IPython.display import Image

def plotImages(images_arr):
    fig, axes = plt.subplots(1, len(images_arr), figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
    
augmented_images = [train_data[0][0][0] for i in range(4)]
plotImages(augmented_images)

NameError: ignored

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(120, 120, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

No. of classes: 10
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 1

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit_generator(
    train_data,
    validation_data=val_data,
    epochs=20
)


Epoch 1/50
1563/1563 [==============================] - 326s 208ms/step - loss: 1.4737 - accuracy: 0.5038 - val_loss: 1.3051 - val_accuracy: 0.5467
Epoch 2/50
1563/1563 [==============================] - 326s 208ms/step - loss: 1.0527 - accuracy: 0.6321 - val_loss: 1.0421 - val_accuracy: 0.6399
Epoch 3/50
1563/1563 [==============================] - 326s 208ms/step - loss: 0.8582 - accuracy: 0.7045 - val_loss: 1.0462 - val_accuracy: 0.6409
Epoch 4/50
1563/1563 [==============================] - 307s 197ms/step - loss: 0.7032 - accuracy: 0.7551 - val_loss: 0.9471 - val_accuracy: 0.6926
Epoch 5/50
1563/1563 [==============================] - 315s 201ms/step - loss: 0.5757 - accuracy: 0.8000 - val_loss: 1.0445 - val_accuracy: 0.6674
Epoch 6/50
1563/1563 [==============================] - 323s 207ms/step - loss: 0.4633 - accuracy: 0.8412 - val_loss: 1.2384 - val_accuracy: 0.6366
Epoch 7/50
1563/1563 [==============================] - 328s 210ms/step - loss: 0.3684 - accuracy: 0.8741 - val_

In [ ]:
plt.plot(history.history['loss'], label='Training Data')
plt.plot(history.history['val_loss'], label='Test Data')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='Training Data')
plt.plot(history.history['val_accuracy'], label='Test Data')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='lower right')
plt.show()